In [ ]:
import json
import os
import requests
from bs4 import BeautifulSoup
import cv2

# Load the JSON files containing the Pokemon data
pokemon_list = []
for i in range(1, 23):
    with open(f'pokedex/pokedex_{i:02d}.json', 'r') as f:
        pokemon_list += json.load(f)

# Create a directory to store the images
if not os.path.exists('images'):
    os.mkdir('images')

# Download the images and store them in the images directory
pokemon_list_with_images = []
for pokemon in pokemon_list:
    name = pokemon['name']
    url = f'https://pokemondb.net/pokedex/{name.lower()}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    image_url = soup.find('a', rel='lightbox')
    if image_url is not None:
        image_url = image_url['href']
        pokemon['image_url'] = image_url

        # Download the image
        image_path = f'images/{name.lower()}.jpg'
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Check if the image is valid
        if os.path.getsize(image_path) > 0:
            # Resize the image to reduce computation time
            image = cv2.imread(image_path)
            image = cv2.resize(image, (100, 100))

            # Add the Pokemon to the list
            pokemon['image_path'] = image_path
            pokemon_list_with_images.append(pokemon)

# Write the Pokemon list with images to a new JSON file
with open('pokedex_with_images.json', 'w') as f:
    json.dump(pokemon_list_with_images, f)


# Similarity check with template-matching (color, grayscale) and ssim

In [3]:
import json
import os
import cv2

# Load the JSON file containing the Pokemon data
with open('pokedex_with_images.json', 'r') as f:
    pokemon_list = json.load(f)

# Load the file containing the pairs of images that have already been compared
resume_file = 'resume.txt'
if os.path.exists(resume_file):
    with open(resume_file, 'r') as f:
        i, j = map(int, f.read().split(','))
else:
    i, j = 0, 0

# Calculate the similarity between each pair of images
with open('similarity.txt', 'a') as f:
    for k in range(i, len(pokemon_list)):
        for l in range(j, len(pokemon_list)):
            pokemon1 = pokemon_list[k]
            pokemon2 = pokemon_list[l]
            image1 = cv2.imread(pokemon1['image_path'])
            image2 = cv2.imread(pokemon2['image_path'])

            # Resize images to the same dimensions
            image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

            similarity = cv2.matchTemplate(image1, image2, cv2.TM_CCOEFF_NORMED)[0][0]
            f.write(f'{pokemon1["name"]},{pokemon2["name"]},{similarity}\n')

            # Save the indices of the last pair of images that were compared
            with open(resume_file, 'w') as f_resume:
                f_resume.write(f'{k},{l}')

        # Reset j to start at the beginning of the next row
        j = 0

    # Delete the resume file when all pairs of images have been compared
    os.remove(resume_file)


In [1]:
import json
import os
import cv2

# Load the JSON file containing the Pokemon data
with open('pokedex_with_images.json', 'r') as f:
    pokemon_list = json.load(f)

# Load the file containing the pairs of images that have already been compared
resume_file = 'resume.txt'
if os.path.exists(resume_file):
    with open(resume_file, 'r') as f:
        i, j = map(int, f.read().split(','))
else:
    i, j = 0, 0

# Calculate the similarity between each pair of images
with open('gs_similarity.txt', 'a') as f:
    for k in range(i, len(pokemon_list)):
        for l in range(j, len(pokemon_list)):
            pokemon1 = pokemon_list[k]
            pokemon2 = pokemon_list[l]
            image1 = cv2.imread(pokemon1['image_path'], cv2.IMREAD_GRAYSCALE)
            image2 = cv2.imread(pokemon2['image_path'], cv2.IMREAD_GRAYSCALE)

            # Resize images to the same dimensions
            image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

            similarity = cv2.matchTemplate(image1, image2, cv2.TM_CCOEFF_NORMED)[0][0]
            f.write(f'{pokemon1["name"]},{pokemon2["name"]},{similarity}\n')

            # Save the indices of the last pair of images that were compared
            with open(resume_file, 'w') as f_resume:
                f_resume.write(f'{k},{l}')

        # Reset j to start at the beginning of the next row
        j = 0

    # Delete the resume file when all pairs of images have been compared
    os.remove(resume_file)

UnboundLocalError: cannot access local variable 'j' where it is not associated with a value

In [7]:
import json
import os
import cv2
from skimage.metrics import structural_similarity as ssim

# Load the JSON file containing the Pokemon data
with open('pokedex_with_images.json', 'r') as f:
    pokemon_list = json.load(f)

# Load the file containing the pairs of images that have already been compared
resume_file = 'resume.txt'
if os.path.exists(resume_file):
    with open(resume_file, 'r') as f:
        i, j = map(int, f.read().split(','))
else:
    i, j = 0, 0

# Calculate the similarity between each pair of images with SSIM
with open('ssim_similarity.txt', 'a') as f:
    for k in range(i, len(pokemon_list)):
        for l in range(j, len(pokemon_list)):
            pokemon1 = pokemon_list[k]
            pokemon2 = pokemon_list[l]
            image1 = cv2.imread(pokemon1['image_path'], cv2.IMREAD_GRAYSCALE)
            image2 = cv2.imread(pokemon2['image_path'], cv2.IMREAD_GRAYSCALE)

            # Resize images to the same dimensions
            image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

            # Calculate the SSIM value between the images
            ssim_score = ssim(image1, image2)

            f.write(f'{pokemon1["name"]},{pokemon2["name"]},{ssim_score}\n')

            # Save the indices of the last pair of images that were compared
            with open(resume_file, 'w') as f_resume:
                f_resume.write(f'{k},{l}')

        # Reset j to start at the beginning of the next row
        j = 0

    # Delete the resume file when all pairs of images have been compared
    os.remove(resume_file)
